# Letters

## Description

Generate an image with the edges of the letters present in this noisy image

## Group 5
- Francisco Macedo Ferreira PG55942
- Júlio José Medeiros Pereira Pinto PG57883
- Ivan Sérgio Rocha Ribeiro PG55950



In [ ]:
import cv2
# to install cv2 use: pip install opencv-python
import matplotlib.pyplot as plt 
import numpy as np

fname = 'Letters-noisy.png'
operations = {}
pipeline_order = []

# Operations

### IDFT


In [ ]:
def compute_idft(dft):
    dft = np.fft.ifftshift(dft)
    img_back = cv2.idft(dft)
    return cv2.magnitude(img_back[:, :, 0], img_back[:, :, 1])

operations['idft'] = compute_idft
print(operations)

### DTF

In [ ]:
def compute_dft(img):
    dft = cv2.dft(img, flags=cv2.DFT_COMPLEX_OUTPUT)
    return np.fft.fftshift(dft)

operations['dft'] = compute_dft
print(operations)

### Inpaint (Black and White)

To remove the black and white pixels we decided to use the function inpaint from OpenCV. It allows us to use different interpolation methods. We decided to use Telea’s Fast Marching Method (`cv2.INPAINT_TELEA`).

In [ ]:
# Inpainting
def inpaint_image(img):
    mask = (img == 0) | (img == 255)
    mask = mask.astype(np.uint8) * 255
    
    return cv2.inpaint(img, mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)

operations['inpaint'] = inpaint_image
print(operations)

### Threshold Filter

In [ ]:
# Apply Threshold Filter
def apply_threshold_filter(dft):
    rows, cols = dft.shape[:2]
    crow, ccol = rows // 2, cols // 2
    
    mask = np.zeros((rows, cols, 2), np.uint8)
    mask[crow - 30:crow + 30, ccol - 30:ccol + 30] = 1
    return dft * mask

operations['threshold'] = apply_threshold_filter
print(operations)

### Gaussian Blurs


In [ ]:
def apply_gaussian_blur(img):
    return cv2.GaussianBlur(img, (5, 5), 0)

operations['gaussian'] = apply_gaussian_blur
print(operations)

### Sobel Magnitude

In [ ]:
def sobel_mag(img):
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=5)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=5)
    
    return np.sqrt(np.power(sobelx,2)+np.power(sobely,2))

operations['sobel'] = sobel_mag
print(operations)

### Contrast

In [ ]:
def apply_contrast(img):
    # Convert to uint8 for histogram equalization
    img_uint8 = img.astype(np.uint8)
    # Apply histogram equalization
    equalized = cv2.equalizeHist(img_uint8)
    return equalized

operations['contrast'] = apply_contrast
print(operations)

## Frequency Filters

### Low-Pass Frequency Filter

We tested with both a circular center as well as rectangular, it didn´t impact the image as much as we would like

In [ ]:
# Apply Low-pass Frequency Filter
def apply_low_pass_filter(dft):
    rows, cols = dft.shape[:2]
    crow, ccol = rows // 2, cols // 2
    
    mask = np.zeros((rows, cols, 2), np.uint8)
    r = 35
    # cv2.circle(mask, (ccol, crow), 30, (1, 1), -1)
    cv2.rectangle(mask, (ccol - r, crow - r + 10), (ccol + r, crow + r - 10), (1, 1), -1)
    return dft * mask

operations['low_pass'] = apply_low_pass_filter
print(operations)

### High-Pass Frequency Filter

It was rather useless but we though we might as well try.

In [ ]:
# Apply High-pass Frequency Filter
def apply_high_pass_filter(dft):
    rows, cols = dft.shape[:2]
    crow, ccol = rows // 2, cols // 2
    
    mask = np.ones((rows, cols, 2), np.uint8)
    cv2.circle(mask, (ccol, crow), 30, (0, 0), -1)
    return dft * mask

operations['high_pass'] = apply_high_pass_filter
print(operations)

## Notch Filters

### Regular Notch Filter

In terms of removing the "checkboard" like effect this was one of the most useful approaches

In [ ]:
# Apply Notch Filter
def apply_notch_filter(dft):
    rows, cols = dft.shape[:2]
    crow, ccol = rows // 2, cols // 2
    
    notch_centers = [(ccol - 30, crow - 20, 10), (ccol + 30, crow + 20, 10), (ccol - 30, crow + 20, 10), (ccol + 30, crow - 20, 10),
                     (ccol + 20, crow, 5), (ccol - 20, crow, 5), (ccol - 5, crow + 10, 5), (ccol + 5, crow - 10, 5)
                     ]
    
    mask = np.ones((rows, cols, 2), np.uint8)
    for x, y, r in notch_centers:
        cv2.circle(mask, (x, y), r, (0, 0), -1)
    return dft * mask

operations['notch'] = apply_notch_filter
print(operations)

### Donut Notch Filter

Just something different we decided to try.


In [ ]:
# Apply Donut Notch Filter
def apply_donut_notch_filter(dft):
    rows, cols = dft.shape[:2]
    crow, ccol = rows // 2, cols // 2
    
    notch_centers = [(ccol - 30, crow - 20, 10, 30), (ccol + 30, crow + 20, 10, 30)]
    
    mask = np.ones((rows, cols, 2), np.uint8)
    
    for x, y, r_inner, r_outer in notch_centers:
        cv2.circle(mask, (x, y), r_outer, (0, 0), -1)
        cv2.circle(mask, (x, y), r_inner, (1, 1), -1)
    return dft * mask

operations['donut'] = apply_donut_notch_filter
print(operations)


### Inverted Notch Filter

Very much like the Low-Pass filter, but this implementations give us a bit more of a way to test it. We tested it with the regular notch_centers array from the Notch Center Function but it looked awful.

In [ ]:

# Apply Inversed Notch Filter
def apply_inversed_notch_filter(dft):
    rows, cols = dft.shape[:2]
    crow, ccol = rows // 2, cols // 2
    
    notch_centers = [(ccol, crow, 30)]

    mask = np.zeros((rows, cols, 2), np.uint8)
    for x, y, r in notch_centers:
        cv2.circle(mask, (x, y), r, (1, 1), -1)
    return dft * mask

operations['inversed_notch'] = apply_inversed_notch_filter
print(operations)

# Helper Functions


### Load Image & Analyzing


In [ ]:

def analyze_image(img):
    hist = cv2.calcHist([img.astype(np.uint8)], [0], None, [256], [0, 256])
    return hist

def apply_analyze_image(img):
    hist = analyze_image(img)
    # Just the histogram
    plt.plot(hist, color='black')
    plt.title('Histogram of Image')
    plt.show()

    return img

operations['analyze'] = apply_analyze_image
print(operations)


def load_image(filename):
    img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    hist = analyze_image(img)
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    axes[0].imshow(img, cmap='gray')
    axes[0].set_title('Original Image')
    axes[0].axis('off')
    
    axes[1].plot(hist, color='black')
    axes[1].set_title('Histogram of Image')
    plt.show()

    return np.float32(img)

### Plots


In [ ]:
def plot_result(data, title):
    if data.ndim == 2:  # Spatial domain
        plt.figure()
        plt.title(title)
        plt.imshow(data, cmap='gray')
        plt.axis('off')
        plt.show()
    elif data.ndim == 3 and data.shape[-1] == 2:  # Frequency domain
        magnitude_spectrum = 20 * np.log(cv2.magnitude(data[:, :, 0], data[:, :, 1]) + 1)
        img = compute_idft(data)
        
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        axes[0].imshow(img, cmap='gray')
        axes[0].set_title('Reconstructed Image')
        axes[0].axis('off')
        
        axes[1].imshow(magnitude_spectrum, cmap='gray')
        axes[1].set_title('Magnitude Spectrum')
        axes[1].axis('off')
        plt.show()

# Main Function

In [ ]:
print(operations)

In [ ]:
# Execute pipeline
data = load_image(fname)

pipeline_order = ["inpaint", "dft", "notch", "low_pass","idft"]
for step in pipeline_order:
    data = operations[step](data)
    plot_result(data, f"Step: {step}")

plot_result(data, "Final Result")